In [274]:
import pandas as pd
import altair as alt
import warnings
import requests
import datetime
import matplotlib.pyplot as plt
import json
import requests
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
alt.renderers.set_embed_options(theme='dark')
pd.set_option('display.max_colwidth', None)

In [275]:
N_PROPOSALS = 8

In [276]:
class AstroDataProvider:
    
    def __init__(self, claim):
        daic_url = "https://terra-api.daic.capital/api/tx/GetRichlistByTokenContract?apiKey=vAp6ysmAXH470YcphYxv&contract_address={}"
        self.votes = '4940a215-6e93-4107-bf08-50574b3e431d'
        self.astro_holders_url =daic_url.format("terra1xj49zyqrwpv5k928jwfpfy2ha668nwdgkwlrg3")
        self.xastro_holders_url =daic_url.format("terra14lpnyzc9z4g3ugr4lhm8s4nle0tq8vcltkhzh7")
        self.claim = claim
        
    def get_from_url(self, url):
        json = requests.get(url).json()
        return json
        
    def load(self):
        self.votes_df = self.claim(self.votes)
        #
        json = self.get_from_url(self.astro_holders_url)['result']['holders']
        self.astro_holders_df = pd.DataFrame(json.values(),json.keys()).reset_index()
        self.astro_holders_df.columns =  ['addr','amount']
        #
        json = self.get_from_url(self.xastro_holders_url)['result']['holders']
        self.xastro_holders_df = pd.DataFrame(json.values(),json.keys()).reset_index()
        self.xastro_holders_df.columns =  ['addr','amount']
        #
        
    def parse_proposal_recap(self):
        votes = self.votes_df.groupby(['proposal_id','vote']).sum().voting_power.reset_index()
        against = votes[votes.vote=='against']
        against.columns = ['proposal_id','against','voting_power_against']
        for_ = votes[votes.vote=='for']
        for_.columns = ['proposal_id','for','voting_power_for']
        votes = against.merge(for_, on='proposal_id')
        votes['delta'] = votes['voting_power_for'] - votes['voting_power_against'] 
        votes['result'] = votes.apply(lambda row: 'passed' if row.delta > 0 else 'failed', axis=1)
        votes['result'] = votes.apply(lambda row: 'passed' if row.delta > 0 else 'failed', axis=1)
        return votes
    
    def parse_top_active_voters(self):
        return dp.votes_df.groupby('voter').agg({'voting_power':'sum','tx_id':'count'})\
                            .sort_values(by=['tx_id','voting_power'], ascending=False)\
                            .head(20)
    def parse_dist_voting_power_per_proposal(self):
        return dp.votes_df[['proposal_id','voting_power']]\
            .pivot(columns='proposal_id',values='voting_power')
    
    def parse_top_voters_per_proposal(self):
        df=[]
        for i in range(1,N_PROPOSALS):
            if(len(df)==0):
                df = dp.votes_df[dp.votes_df.proposal_id==1].sort_values(by='voting_power', ascending=False).head(10)
            else:
                df = df.append(dp.votes_df[dp.votes_df.proposal_id==i].sort_values(by='voting_power', ascending=False).head(10))
        return df
    
    def parse_votes_over_time(self):
        df_ = []
        for i in range(1,N_PROPOSALS):
            df = self.votes_df[dp.votes_df.proposal_id==i].groupby(['hr','proposal_id','vote']).voting_power.sum().reset_index()
            df_for = df[df.vote=='for']
            df_for.columns = ['hr','proposal_id','vote_for','voting_power_for']
            df_against = df[df.vote=='against']
            df_against.columns = ['hr','proposal_id','vote_against','voting_power_against']
            df = df_for.merge(df_against, on=['hr','proposal_id'], how='outer')
            df.vote_against = df.vote_against.fillna('against')
            df.vote_for = df.vote_for.fillna('for')
            df.voting_power_against = df.voting_power_against.fillna(0)
            df.voting_power_for = df.voting_power_for.fillna(0)
            df['voting_power_for_cumsum'] = df.sort_values(by=['hr']).voting_power_for.cumsum()
            df['voting_power_against_cumsum'] = df.sort_values(by=['hr']).voting_power_against.cumsum()
            if(len(df_)==0):
                df_ = df
            else:
                df_ = df_.append(df)
        return df_
    
    def parse_voting_power_cumulative(self):
        df_ = []
        top_various_prop = []
        for i in range(1,N_PROPOSALS):
            df = dp.votes_df
            df = df[df.proposal_id==i]
            df['voting_power_perc'] = df.voting_power/df.voting_power.sum()*100
            df = df.sort_values(by='voting_power_perc', ascending=False)
            df['voting_power_cumsum'] = df.voting_power_perc.fillna(0).cumsum()
            df['n_addresses'] = np.arange(len(df))+1
            df['n_addresses_perc'] = (1-df.n_addresses/df.n_addresses.max())*100
            assert (int(df.voting_power_cumsum.max()) in [100,99]), 'Total % should give 100'
            top_various_prop.append((i,len(df[df.voting_power_cumsum<50])+1))
            if(len(df_)==0):
                    df_ = df
            else:
                df_ = df_.append(df)
        top_various_prop_df = pd.DataFrame(top_various_prop)
        top_various_prop_df.columns = ['proposal_id','n_for_majority']
        return df_, top_various_prop_df
    
    def parse(self):
        df = self.votes_df
        df['block_timestamp'] = df.block_timestamp.astype('datetime64[ms]')
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-7] if len(x) == 26 else x)
        self.votes_df = df
        #
        dp.votes_df['hr'] = dp.votes_df.block_timestamp.apply(str).str[:-5] + '00'
        dp.votes_df['day'] = dp.votes_df.block_timestamp.apply(str).str[:-9]
        self.astro_holders_df.amount = self.astro_holders_df.amount/1000000
        self.proposal_recap = self.parse_proposal_recap()
        self.top_active_voters = self.parse_top_active_voters()
        self.dist_voting_power_per_proposal = self.parse_dist_voting_power_per_proposal()
        self.top_voters_per_proposal = self.parse_top_voters_per_proposal()
        self.votes_over_time = self.parse_votes_over_time()
        self.voting_power_cumulative, self.majority_per_vote = self.parse_voting_power_cumulative()
        
    def to_file(self, path='../data'):
        self.votes_df.to_json(f"{path}/votes_df",orient='records')
        self.astro_holders_df.to_json(f"{path}/astro_holders_df",orient='records')
        self.proposal_recap.to_json(f"{path}/proposal_recap", orient='records')
        self.top_active_voters.to_json(f"{path}/top_active_voters",orient='records')
        self.dist_voting_power_per_proposal.to_json(f"{path}/dist_voting_power_per_proposal",orient='records')
        self.top_voters_per_proposal.to_json(f"{path}/top_voters_per_proposal",orient='records')
        self.votes_over_time.to_json(f"{path}/votes_over_time",orient='records')
        self.voting_power_cumulative.to_json(f"{path}/voting_power_cumulative",orient='records')
        self.majority_per_vote.to_json(f"{path}/majority_per_vote",orient='records')
        
    def read_file(self):
        url = 'https://raw.githubusercontent.com/IncioMan/astroport_governance/master/data/{}'
        self.votes_df =  pd.read_json(url.format('votes_df'))
        self.astro_holders_df =  pd.read_json(url.format('astro_holders_df'))
        self.proposal_recap =  pd.read_json(url.format('proposal_recap'))
        self.top_active_voters =  pd.read_json(url.format('top_active_voters'))
        self.dist_voting_power_per_proposal =  pd.read_json(url.format('dist_voting_power_per_proposal'))
        self.top_voters_per_proposal =  pd.read_json(url.format('top_voters_per_proposal'))
        self.votes_over_time =  pd.read_json(url.format('votes_over_time'))
        self.voting_power_cumulative =  pd.read_json(url.format('voting_power_cumulative'))
        self.majority_per_vote =  pd.read_json(url.format('majority_per_vote'))


In [277]:
def claim(claim_hash):
    df = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"])
    df.columns = [c.lower() for c in df.columns]
    return df

In [278]:
dp = AstroDataProvider(claim)
dp.load()
dp.parse()
dp.to_file()
#dp.read_file()

/var/folders/pd/3vx1d3qj29z1l1tm70p573s80000gn/T/ipykernel_23303/1063528383.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['voting_power_perc'] = df.voting_power/df.voting_power.sum()*100
/var/folders/pd/3vx1d3qj29z1l1tm70p573s80000gn/T/ipykernel_23303/1063528383.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['voting_power_perc'] = df.voting_power/df.voting_power.sum()*100
/var/folders/pd/3vx1d3qj29z1l1tm70p573s80000gn/T/ipykernel_23303/1063528383.py:82: SettingWithCopyWarning: 
A value is

dp = AstroDataProvider(claim)
#dp.load()
#dp.parse()
#dp.to_file()
dp.read_file()

In [279]:
dp.voting_power_cumulative

,block_timestamp,tx_id,action,proposal_id,vote,voter,voting_power,hr,day,voting_power_perc,voting_power_cumsum,n_addresses,n_addresses_perc
6488,2022-04-11 19:58:22,CA97CC66C576EC234AE0992CD4068DCA5A896E3409043E9E2BDE084ACFD88C10,cast_vote,1,for,terra1zaqeperrwghqlsa9yykzsjaets54mtq0u6kl60,1.500000e+13,2022-04-11 19:00,2022-04-11,3.187712e+01,31.877116,1,99.951784
581,2022-04-12 07:51:31,FE7F2727A08EE6864240616B4A7A7962BBE8A9C332693DACF47C13D45335E66E,cast_vote,1,for,terra1pq02fnrm68x6kcv2lhgvyetjelps550w3pq6m2,5.000000e+12,2022-04-12 07:00,2022-04-12,1.062571e+01,42.502822,2,99.903568
1609,2022-04-12 14:19:16,6F669E183D4AE72EACE06046DE8FCEE6EE5C1D53B4C4C2B8F18EF09D468FC05F,cast_vote,1,for,terra1jyxljeq865wsndutyyr0vrzzq9tj3fa6rmhgrl,3.764988e+12,2022-04-12 14:00,2022-04-12,8.001131e+00,50.503952,3,99.855352
6213,2022-04-15 16:57:50,2B02AEA78FAC0A7CF318288D4A22768280C6611EAE765E00D4321121891A74DA,cast_vote,1,for,terra17j6tjl2zxd0lugwz3vvsvjcl0z34kh9hqaa63l,2.500000e+12,2022-04-15 16:00,2022-04-15,5.312853e+00,55.816805,4,99.807136
7720,2022-04-11 21:24:01,DA9D0C81F1E5A4282E21CB479EDB27719189D28C158B2C6CF6B77979751DBD5F,cast_vote,1,for,terra1gn53cj0v8kvwxqg867e3mu9f3q9yzskmfgnvla,2.007961e+12,2022-04-11 21:00,2022-04-11,4.267201e+00,60.084006,5,99.758920
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5741,2022-04-28 04:00:21,1B956DC527CE5CDF8E8139E6EF32101D1AFA4BC5A288D799FBB67404B4CE3166,cast_vote,7,for,terra15sm4h54hfz6r9rhfwed7agg73m6w0v4a55g6uz,1.245280e+05,2022-04-28 04:00,2022-04-28,2.205815e-07,100.000000,1019,0.391007
4323,2022-04-27 04:34:31,76FF9BAEDD9911C68C50362563B4E2A9E5465F072BBA5F7F842EEB45E40E0514,cast_vote,7,for,terra1zml98h94nqkyc47egx9dcru55s99du6ckjpmjq,1.193470e+05,2022-04-27 04:00,2022-04-27,2.114042e-07,100.000000,1020,0.293255
2782,2022-04-29 13:33:59,F4BF03DE7810B2F467E7582B10F5943C3B2BC2D4816593E2CA168861CEA70320,cast_vote,7,for,terra1ln5dmnj2xgm42v6n9w39vyhfmmv9f8azfv7xrh,5.576600e+04,2022-04-29 13:00,2022-04-29,9.878060e-08,100.000000,1021,0.195503
2017,2022-04-28 19:07:19,1316FAEA53C5EE4CF43A351127AF088B49E714C731943B83F0E0AA889D375057,cast_vote,7,for,terra1x7dvfvqf43llv67d3x749x6d0yfda0jwhg3j2a,2.951100e+04,2022-04-28 19:00,2022-04-28,5.227404e-08,100.000000,1022,0.097752
